<a target="_blank" href="https://colab.research.google.com/github/ZHAW-ZAV/TSO-FS25/blob/main/02_python_viz/02_11_exercise_co2.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
import sys
import os

IN_COLAB = "google.colab" in sys.modules

file_id_co2 = "1-FxcOXUXMrXZzDzeLgarKoQfa2QN_sXO"
file_id_capitals = "1U9IbkK9cT94bJlJ1Ie1VLfqWFv5HOZBM"

if IN_COLAB:
    path_to_co2 = "/content/data/co2_emmissions_by_state.csv"
    os.makedirs(os.path.dirname(path_to_co2), exist_ok=True)
    !gdown "https://drive.google.com/uc?id={file_id_co2}" -O "{path_to_co2}"

    path_to_capitals = "/content/data/capitals.csv"
    os.makedirs(os.path.dirname(path_to_capitals), exist_ok=True)
    !gdown "https://drive.google.com/uc?id={file_id_capitals}" -O "{path_to_capitals}"

    # Unfortunately, Colab does not come with the Kaleido library installed.
    !pip install -q kaleido

else:
    import gdown

    url = f"https://drive.google.com/uc?id={file_id_co2}"
    path_to_co2 = "data/co2_emmissions_by_state.csv"
    os.makedirs(os.path.dirname(path_to_co2), exist_ok=True)
    gdown.download(url, path_to_co2, quiet=False)

    url = f"https://drive.google.com/uc?id={file_id_capitals}"
    path_to_capitals = "data/capitals.csv"
    os.makedirs(os.path.dirname(path_to_capitals), exist_ok=True)
    gdown.download(url, path_to_capitals, quiet=False)


The code above loads the data, don't modify.

---------------

***Notebook starts here***

# Analyzing CO2 emissions

This exercise uses the same dataset as in the previous week. The description of the dataset is repeated below for convenience.

In this exercise, we will analyze the CO2 emissions of different countries. The data is provided in the file `co2_emmissions_by_state.csv`, starting from the year 2010. The data contains the following columns:
- `YEAR`: Year of the data
- `MONTH`: Month of the data, from 1 to 12
- `STATE_NAME`: Name of the country
- `STATE_CODE`: Two letter code of the country
- `CO2_QTY_TONNES`: CO2 emissions in metric tons
- `TF`: Number of flights

This dataset is provided by EUROCONTROL and publicly available [here](https://ansperformance.eu/data/) and the description [here](https://ansperformance.eu/reference/dataset/emissions/).

The data contains the calculated emissions from all IFR flights departing from a EUROCONTROL member state. For instance, the total CO2 emissions for the full trajectory of a flight from Brussels to New York would be included in the results for Belgium. A flight from New York to Brussels would however NOT be included in the results for Belgium.

## Tasks
This exercise is guided and you will be asked to implement the visualizations step by step. You are free to use Plotly Express and/or Graph Objects. The tasks are:
1. Load the data and prepare the data for visualization
2. Overview of the CO2 emissions
3. Analyze the CO2 emissions of the countries

In [ ]:
from plotly import graph_objects as go
from plotly import express as px
import pandas as pd

## Line plots, bar plots, and pie charts

In [ ]:
# Load the data
co2 = pd.read_csv(path_to_co2)

In [ ]:
# Let's start with an overview of the total CO2 emissions per year.
# For this, we first need to aggregate the data by year.
#
# Use the groupby method to group the data by year and sum the emissions.

co2_by_year = co2.groupby("YEAR")["CO2_QTY_TONNES"].sum().reset_index()
co2_by_year.head()

In [ ]:
# Now we can create a visualization CO2 emissions per year.
# Create a line chart for the data. Make sure to include a title and label the axes.

# with PX
fig = px.line(co2_by_year, x="YEAR", y="CO2_QTY_TONNES", markers=True)
fig.update_layout(title="CO2 emissions per year")
fig.update_xaxes(title="Year")
fig.update_yaxes(title="CO2 emissions (tonnes)")
fig.show()

# with GO
fig = go.Figure()
fig.add_trace(go.Scatter(x=co2_by_year["YEAR"], y=co2_by_year["CO2_QTY_TONNES"]))
fig.update_layout(title="CO2 emissions per year")
fig.update_xaxes(title="Year")
fig.update_yaxes(title="CO2 emissions (tonnes)")
fig.show()

In [ ]:
# We would like to visualize the CO2 emissions per year for each state.
# For this, we need to group the data by year and state.

co2_by_year_state = (
    co2.groupby(["YEAR", "STATE_NAME"])["CO2_QTY_TONNES"].sum().reset_index()
)
co2_by_year_state.head()

In [ ]:
# Now we can create a visualization of the CO2 emissions per year for each state.
# Create a stacked bar chart for the data. Make sure to include a title and label the axes.

# with PX
fig = px.bar(co2_by_year_state, x="YEAR", y="CO2_QTY_TONNES", color="STATE_NAME")
fig.update_layout(title="CO2 emissions per year by state")
fig.update_xaxes(title="Year")
fig.update_yaxes(title="CO2 emissions (tonnes)")
fig.show()

# with GO
fig = go.Figure()
state_names = co2_by_year_state["STATE_NAME"].unique()
for state in state_names:
    data = co2_by_year_state.query("STATE_NAME == @state")
    fig.add_trace(go.Bar(x=data["YEAR"], y=data["CO2_QTY_TONNES"], name=state))
fig.update_layout(title="CO2 emissions per year by state", barmode="stack")
fig.update_xaxes(title="Year")
fig.update_yaxes(title="CO2 emissions (tonnes)")
fig.show()

In [ ]:
# Next, we create a pie chart to show the distribution of CO2 emissions by state for all years.
# As before, group the data first, this time by state and sum the emissions.

co2_by_state = co2.groupby("STATE_NAME")["CO2_QTY_TONNES"].sum().reset_index()
co2_by_state.head()

In [ ]:
# Now we can create a visualization of the distribution of CO2 emissions by state.
# Create a pie chart for the data. Make sure to include a title.
#
# Hints:
# - Use the online documentation to find the correct arguments for the pie chart.
# - If you want to suppress the value labels on the pie chart (the value in %), add the following line:
#   fig.update_traces(textinfo="none")

# with PX
fig = px.pie(co2_by_state, names="STATE_NAME", values="CO2_QTY_TONNES")
fig.update_layout(title="CO2 emissions by state")
fig.update_traces(textinfo="none")
fig.show()

# with GO
fig = go.Figure()
fig.add_trace(
    go.Pie(labels=co2_by_state["STATE_NAME"], values=co2_by_state["CO2_QTY_TONNES"])
)
fig.update_layout(title="CO2 emissions by state")
fig.update_traces(textinfo="none")
fig.show()

## Plotting on a map

Now we want to do a fancy plot on a map. The size of the marker should be proportional to the CO2 emissions of the country.

In [ ]:
# Read the CSV file containing the coordinates of the capital cities.
# We need to do a bit of preprocessing to make the country names compatible with the CO2 data.
capitals = pd.read_csv(path_to_capitals)

capitals["CountryName"] = capitals["CountryName"].replace("Turkey", "TÜRKIYE")
capitals["CountryName"] = capitals["CountryName"].str.upper()
capitals.head()

In [ ]:
# Merge the CO2 data with the capital cities data.
# To make our lives easier, we will use the already aggregated CO2 data by state.
# The key for the merge is the state name. In the CO2 data, the state name is in the column "STATE_NAME", and
# in the capital cities data, it is in the column "CountryName".

co2_with_cap = co2_by_state.merge(
    capitals, left_on="STATE_NAME", right_on="CountryName", how="left"
)
co2_with_cap.head()

In [ ]:
# Let's visualize the CO2 emissions on a map.
# Create a plot with round markers. The marker size should be proportional to the CO2 emissions.
# Make sure to include a title and adjust the center and zoom level of the map so that all markers are visible.
#
# Hints:
# - You can use either Plotly Express or Plotly Graph Objects, but it's quite a bit easier with Plotly Express.
# - Use the CapitalLatitude and CapitalLongitude columns for the location.
# - You can modify the marker size in Plotly Express with the size_max argument. Play around with the value to find
#   size that looks good to you.


# with PX
fig = px.scatter_map(
    co2_with_cap,
    lat="CapitalLatitude",
    lon="CapitalLongitude",
    size="CO2_QTY_TONNES",
    hover_name="STATE_NAME",
    size_max=40,
)
fig.update_layout(
    title="CO2 emissions by state",
    height=800,
    width=800,
    map=dict(
        center=dict(
            lat=co2_with_cap["CapitalLatitude"].mean(),
            lon=co2_with_cap["CapitalLongitude"].mean(),
        ),
        zoom=2.5,
    ),
)
fig.show()

# ADVANCED
# with GO

# This is a bit tricky because go.Scattermapbox expects the size in pixels, but we have the size in tonnes.
# We need to compute the size in pixels based on the size in tonnes first.

# we will use the following formula to compute the size in pixels:
# size in pixels = (size in tonnes / max size in tonnes) * max size in pixels
max_size = co2_with_cap["CO2_QTY_TONNES"].max()
max_size_pixels = 60
co2_with_cap["size"] = (co2_with_cap["CO2_QTY_TONNES"] / max_size) * max_size_pixels

fig = go.Figure()
fig.add_trace(
    go.Scattermap(
        lat=co2_with_cap["CapitalLatitude"],
        lon=co2_with_cap["CapitalLongitude"],
        mode="markers",
        marker_size=co2_with_cap["size"],
        text=co2_with_cap["STATE_NAME"],
    )
)
fig.update_layout(
    title="CO2 emissions by state",
    height=800,
    width=800,
    map=dict(
        center=dict(
            lat=co2_with_cap["CapitalLatitude"].mean(),
            lon=co2_with_cap["CapitalLongitude"].mean(),
        ),
        zoom=2.5,
    ),
)
fig.show()

## Export
Since the figure above is so pretty, we want to export it.

In [ ]:
# Export the previous figure.
# - Export to a PNG file. Open the file in an image viewer to verify that the export was successful.
# - Export to a HTML file. Open the file in a browser to verify that the export was successful.

fig.write_image("co2_per_country.png")
fig.write_html("co2_per_country.html")